In [2]:
import pandas as pd
import sqlalchemy as sql

In [3]:
recessions={
    'savings_and_loans_crisis': pd.date_range('1990-07', '1991-03'), 
    'dot_com_bust': pd.date_range('2001-03', '2001-11'), 
    'great_recession': pd.date_range('2007-12', '2009-06'), 
    'covid': pd.date_range('2020-04', '2020-06'), 
    # subject to change
    'current': pd.date_range('2022-01', '2022-08-09')
}

In [4]:
engine=sql.create_engine('sqlite:///recession_analysis.db')

In [5]:
sql_query=f'select * from price_history'
price_history=pd.read_sql(sql_query, con=engine, index_col='Date', parse_dates='Date')

price_history=price_history.pivot(columns='ticker', values='Close').pct_change()
price_history.head()

ticker,IYT,MSCI,SPY,VENAX,VGHCX,^IXIC
Date,,,,,,
1971-02-05,NaN,NaN,NaN,NaN,NaN,NaN
1971-02-08,NaN,NaN,NaN,NaN,NaN,0.008400
1971-02-09,NaN,NaN,NaN,NaN,NaN,-0.000793
1971-02-10,NaN,NaN,NaN,NaN,NaN,-0.000695
1971-02-11,NaN,NaN,NaN,NaN,NaN,0.007548


In [22]:
post_recessions={}
for recession_name, recession_dates in recessions.items(): 
    post_recessions[recession_name]=pd.date_range(start=recession_dates[-1], periods=252)

In [23]:
recession_return={}
for recession_name, recession_dates in post_recessions.items(): 
    recession_df=price_history.loc[price_history.index.isin(recession_dates)]
    recession_return[recession_name]=(recession_df+1).cumprod().iloc[-1]

In [24]:
pd.DataFrame(recession_return).T

ticker,IYT,MSCI,SPY,VENAX,VGHCX,^IXIC
savings_and_loans_crisis,NaN,NaN,NaN,NaN,1.187798,1.203443
dot_com_bust,NaN,NaN,0.879264,NaN,0.859968,0.796361
great_recession,1.212385,1.353079,1.171011,1.076732,1.189376,1.206720
covid,1.423212,1.312468,1.290226,1.190851,1.099552,1.460115
current,0.990057,0.986283,0.994673,1.000000,1.000000,0.987189


In [1]:
# Visually looking at the variance graph, Dot Com Bust most resembles current recession which would allude to SPY being the best investment.
# However also indicating that return after 252 days we are still at a negative return and therefore may need more time to recover. 
# Overall MSCI representing Fianancial Sector and ^IXIC representing Tech Sector did well overall recessions where data was available.